In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
tickers = ['JNK', 'TLT']
ydata = yf.Tickers(tickers).download(period="max", auto_adjust=True)   # get dividend adjusted price

In [ ]:
data = (   # put data in long form
    ydata
    .dropna()
    .stack(1, future_stack=True)
    .reset_index()
    [['Date', 'Ticker', 'Close']]
    .sort_values(by=['Date', 'Ticker'])
)

data.head(5)

In [ ]:
# download historical data from Yahoo Finance to csv, file name data_<tickers>_2002-07-30_D.csv
data.to_csv('data_JNK_TLT_20XX-XX-XX_D.csv', index=False)

In [ ]:
# read data from csv
data = pd.read_csv('data_JNK_TLT_20XX-XX-XX_D.csv')
data['Date'] = pd.to_datetime(data['Date'])
print(data.info())
print(data.head(5))

In [ ]:
# Add year-month label (end of month date)

# Convert Date column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Create a new column 'ym' representing the last day of each month
data['ym'] = data['Date'] + pd.offsets.MonthEnd(0)

data.head()

In [ ]:
# Create trading-day counter for each Ticker and month

# Sort data to ensure correct order
data = data.sort_values(['Ticker', 'Date']).copy()

# Count trading days within each (Ticker, ym) group
data['td'] = data.groupby(['Ticker', 'ym']).cumcount() + 1

data.head()

In [ ]:
#  Calculate daily simple returns

# Calculate percentage change of 'Close' within each ticker
data['ret'] = data.groupby('Ticker')['Close'].pct_change()

data.head()

In [ ]:
# Filter dates and keep only needed columns

# Keep data from August 2002 onwards
data = data[data['Date'] >= pd.Timestamp(2002, 8, 1)]

# Keep only useful columns
data = data[['Date', 'ym', 'Ticker', 'Close', 'ret', 'td']]

data.head()

In [ ]:
# Create start/end/month flags and conditional returns

N = 15  # number of days for start of month
M = 5   # number of days for next start of month

# Flags: 1 if condition true, 0 otherwise
data['som_flag'] = (data['td'] <= N).astype(int)
data['eom_flag'] = (data['td'] > N).astype(int)
data['nsm_flag'] = (data['td'] <= M).astype(int)

# Conditional returns (return only within flag period)
data['ret_som'] = data['ret'] * data['som_flag']
data['ret_eom'] = data['ret'] * data['eom_flag']
data['ret_nsm'] = data['ret'] * data['nsm_flag']

data.head()

In [ ]:
# Keep only relevant return columns
data_flags = data[['Date', 'ym', 'Ticker', 'ret_som', 'ret_eom', 'ret_nsm']]

data_flags.head()

In [ ]:
# Pivot to wide format (one column per Ticker)

# Create wide format table: columns for each Ticker and return type
pivoted = data_flags.pivot(index=['Date', 'ym'], columns='Ticker')

# Flatten multi-level column names (e.g. ret_som_SPY)
pivoted.columns = [f"{col[1]}_{col[0]}" for col in pivoted.columns]

pivoted = pivoted.reset_index()

pivoted.head()

In [ ]:
# create a copy of data to be later used for the backtest analysis (8 PL streams in the backtesting)
backtest_data = pivoted.copy()

In [ ]:
# Compute relative returns and aggregate monthly

# Calculate combined and relative returns
pivoted['JKN_ret'] = pivoted['JKN_ret_som'] + pivoted['JKN_ret_eom']
pivoted['JKN_TLT_som'] = pivoted['JKN_ret_som'] - pivoted['TLT_ret_som']
pivoted['JKN_TLT_eom'] = pivoted['JKN_ret_eom'] - pivoted['TLT_ret_eom']
pivoted['JKN_TLT_nsm'] = pivoted['JKN_ret_nsm'] - pivoted['TLT_ret_nsm']

# Select relevant columns
returns = pivoted[['ym', 'JKN_ret', 'JKN_TLT_som', 'JKN_TLT_eom', 'JKN_TLT_nsm']]

# Aggregate monthly returns using compounding formula
monthly = returns.groupby('ym').apply(lambda x: (1 + x).prod() - 1)

monthly.head()

In [ ]:
return_data = monthly.copy()